In [6]:
from urllib import request
from bs4 import BeautifulSoup
import re

In [24]:
LLVM_RELEASE_PAGE = r"https://github.com/llvm/llvm-project/releases"

with request.urlopen(LLVM_RELEASE_PAGE) as req:
    page = str(req.read())

In [158]:
type(page)

str

In [124]:
soup = BeautifulSoup(page, "html.parser")

In [126]:
len(soup.find_all("section"))

10

In [137]:
for section in soup.find_all("section"):
    print(section.find("h2", attrs = {"class": "sr-only"}).text)

LLVM 16.0.0-rc3
LLVM 16.0.0-rc2
LLVM 16.0.0-rc1
LLVM 15.0.7
LLVM 15.0.6
LLVM 15.0.5
LLVM 15.0.4
LLVM 15.0.3
LLVM 15.0.2
LLVM 15.0.1


In [139]:
for section in soup.find_all("section"):
    print(section.find_all("span", attrs = {"class": "f3 text-bold d-inline mr-3"}))

[<span class="f3 text-bold d-inline mr-3" data-view-component="true">Assets</span>]
[<span class="f3 text-bold d-inline mr-3" data-view-component="true">Assets</span>]
[<span class="f3 text-bold d-inline mr-3" data-view-component="true">Assets</span>]
[<span class="f3 text-bold d-inline mr-3" data-view-component="true">Assets</span>]
[<span class="f3 text-bold d-inline mr-3" data-view-component="true">Assets</span>]
[<span class="f3 text-bold d-inline mr-3" data-view-component="true">Assets</span>]
[<span class="f3 text-bold d-inline mr-3" data-view-component="true">Assets</span>]
[<span class="f3 text-bold d-inline mr-3" data-view-component="true">Assets</span>]
[<span class="f3 text-bold d-inline mr-3" data-view-component="true">Assets</span>]
[<span class="f3 text-bold d-inline mr-3" data-view-component="true">Assets</span>]


In [146]:
# the latest stable release is partially expanded automatically!

for section in soup.find_all("section"):
    include_fragment = section.find("include-fragment", attrs = {"loading": "lazy"})
    if include_fragment:
        print(include_fragment.get("src"))

https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-16.0.0-rc3
https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-16.0.0-rc2
https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-16.0.0-rc1
https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-15.0.6
https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-15.0.5
https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-15.0.4
https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-15.0.3
https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-15.0.2
https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-15.0.1


In [149]:
for section in soup.find_all("section"):
    include_fragment = section.find("include-fragment", attrs = {"loading": "lazy"})
    if not include_fragment:
        print(section.find("include-fragment", attrs = {"class": "js-truncated-assets-fragment"}).get("data-deferred-src"))

https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-15.0.7


In [150]:
win64 = re.compile(r"LLVM(.)*win64.exe")

In [ ]:
def extract_llvm_release_links(html_document: str) -> list[str]:
    
    """
    A function that takes GitHub LLVM releases HTML page as a string and returns
    a list of links to actually access the download URIs.
    Requires re and BeautifulSoup.
    """
    
    if not isinstance(html_document, str):
        raise TypeError("Incompatible types. Argument must be of string <class 'str'> type.")
    
    try:
        from bs4 import BeautifulSoup
    except ImportError:
        raise ImportError("Unavailable application dependency: BeautifulSoup.")
        
    links: list[str] = list()
    
    soup = BeautifulSoup(html_document)

In [154]:
isinstance("89", str)

True

In [157]:
print(type("eqf"))

<class 'str'>
